In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date, time, timedelta

In [2]:
#Define write function for csv:
def write_to_csv(df, write_path, sep=',', encoding='latin-1', save_index=False):
    try:
        df.to_csv(write_path, sep=sep, encoding=encoding, index=save_index)
        print(f"Writing at {write_path} complete!")
    except Exception as e:
        print(f"Error writing at {write_path}, Error: {e}")
#end

#Define write function for zipped csv:
def write_to_csv_zip(df, write_path, sep=',', encoding='latin-1'):
    try:
        paths = write_path.split('/')
        filename = paths[len(paths) - 1] if len(paths) > 0 else None
        if filename is None:
            raise Exception("filename is None!")
        write_path = write_path.replace('.csv', '.zip')
        df.to_csv(write_path
                  , sep=sep
                  , encoding=encoding
                  , index=False
                  , compression={'method':'zip','archive_name':filename})
        print(f"Writing at {write_path} complete!")
    except Exception as e:
        print(f"Error writing at {write_path}, Error: {e}")
#end

In [5]:
##Test: writing *.csv as *.zip
write_to_csv_zip(pd.DataFrame({'Name':['Alice','Sophia','Júlia'], 'Age':[28, 24, 21]})
                 , "../resources/example_csv_as_zipped_a.csv")

Writing at ../resources/example_csv_as_zipped_a.zip complete!


In [31]:
str_dt = "10_2023" # "%m_%Y"
#month_year : need to find which qtr of this month of year:
##
print(datetime.now())
print(datetime.strptime(str_dt, "%m_%Y"))
##
p_dtime = datetime.strptime(str_dt, "%m_%Y")
print(f"year:{p_dtime.year}, month:{p_dtime.month}")

2024-05-13 14:57:34.678979
2023-10-01 00:00:00
year:2023, month:10


In [3]:
import math

##Parse datetime using strptime():
def getQtr(dtime):
    return int(math.ceil(dtime.month) / 3) + 1
#end

def getQtrFromStrptime(str_dtime, f_format = "%Y-%m-%d"):
    try:
        dtime = datetime.strptime(str_dtime, f_format)
        return int(math.ceil(dtime.month) / 3) + 1
    except Exception as e:
        pass
    return 0
#end

def getYearFromStrptime(str_dtime, f_format = "%Y-%m-%d"):
    try:
        dtime = datetime.strptime(str_dtime, f_format)
        return dtime.year
    except Exception as e:
        pass
    return date.today().year
#end

def getIntFromStrptime(str_dtime, regx = '-'):
    try:
        vals = str_dtime.strip().split(regx)
        vals.reverse()
        return int(''.join(vals))
    except Exception as e:
        pass
    return 0
#end

def strftime(str_date, frm_format = "%Y-%m-%d", to_format = "%d/%m/%Y", split_by = ' '):
    try:
        splited = str_date.split(split_by)
        dt = datetime.strptime(splited[0], frm_format)
        return dt.strftime(to_format)
    except Exception as e:
        pass
    return ""
#end

In [33]:
##Test datetime parsing: 
for str_dtime in ['02_2023', '04_2023', '08_2023', '10_2023']:
    m_dtime = datetime.strptime(str_dtime, "%m_%Y")
    qtr = getQtr(m_dtime)
    print(f"{m_dtime} -> Qtr: {qtr}")
#end

2023-02-01 00:00:00 -> Qtr: 1
2023-04-01 00:00:00 -> Qtr: 2
2023-08-01 00:00:00 -> Qtr: 3
2023-10-01 00:00:00 -> Qtr: 4


In [5]:
##Pandas dataframe practice:
#Create a pandas dataframe with dictionary data:
today = date.today()
raw_data = {'Name':['Alice', 'Sophia', 'Júlia']
            , 'Age':[28, 24, 20]
            , 'Date':[today - timedelta(days=300), today - timedelta(days=600), today - timedelta(days=900)]}
df = pd.DataFrame(raw_data)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    3 non-null      object
 1   Age     3 non-null      int64 
 2   Date    3 non-null      object
dtypes: int64(1), object(2)
memory usage: 204.0+ bytes


In [35]:
#Subset Dataframe with selective columns: 
df[['Name','Date']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    3 non-null      object
 1   Date    3 non-null      object
dtypes: object(2)
memory usage: 180.0+ bytes


In [36]:
#All values from Date column as List/Series: 
df['Date'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 3 entries, 0 to 2
Series name: Date
Non-Null Count  Dtype 
--------------  ----- 
3 non-null      object
dtypes: object(1)
memory usage: 156.0+ bytes


In [37]:
#All values from a selected row as List/Series:
df.loc[0].info()

<class 'pandas.core.series.Series'>
Index: 3 entries, Name to Date
Series name: 0
Non-Null Count  Dtype 
--------------  ----- 
3 non-null      object
dtypes: object(1)
memory usage: 156.0+ bytes


In [6]:
#A specific cell value from a row and a column as a Scalar:
print(f"Name:{df.loc[1, 'Name']}; Age:{df.loc[1, 'Age']}")

Name:Sophia; Age:24


In [41]:
##Show all column-names:
#print(df.columns.tolist())
list(df.columns)

['Name', 'Age', 'Date']

In [73]:
##Show df index as RangeIndex:
df.index

RangeIndex(start=0, stop=3, step=1)

In [75]:
##Pick subset of idx:
df.index[[0, 1]]

Index([0, 1], dtype='int64')

In [7]:
##Loop-through df.index:
for idx in df.index:
    print(f"{idx} : {df.loc[idx, 'Name']}")
#end

0 : Alice
1 : Sophia
2 : Júlia


In [8]:
#Add new Qtr column to df:
df['Qtr'] = df['Date'].apply(lambda x: getQtr(x))
df.head()

,Name,Age,Date,Qtr
0,Alice,28,2023-07-21,3
1,Sophia,24,2022-09-24,4
2,Júlia,20,2021-11-28,4


In [ ]:
##Pandas.DataFrame: apply() vs map() vs applymap():
"""
Definition

    map is defined on Series only
    applymap is defined on DataFrames only
    apply is defined on both

Input argument

    map accepts dict, Series, or callable
    applymap and apply accept callable only

Behavior

    map is elementwise for Series
    applymap is elementwise for DataFrames
    apply also works elementwise but is suited to more complex operations and aggregation. The behaviour and return value depends on the function.

Use case (the most important difference)

    map is meant for mapping values from one domain to another, so is optimised for performance, e.g.,

    df['A'].map({1:'a', 2:'b', 3:'c'})

    applymap is good for elementwise transformations across multiple rows/columns, e.g.,

    df[['A', 'B', 'C']].applymap(str.strip)

    apply is for applying any function that cannot be vectorised, e.g.,

    df['sentences'].apply(nltk.sent_tokenize)

Also see: 'When should I (not) want to use pandas apply() in my code?
"""
"""
Summarising
	                    map 	applymap 	apply
Defined on Series? 	    Yes 	   No 	     Yes
Defined on DataFrame?   No 	       Yes 	     Yes
Elementwise?            Yes        Yes       Yes
Aggregation?            No         No        Yes
Returns               Series    DataFrame  Scalar,Series,DataFrame
"""

In [9]:
#Apply: can operate on series, scalar or dataframe
df['Fullname'] = df['Name'].apply(lambda nm: f"{nm} Engel")
df.head()

,Name,Age,Date,Qtr,Fullname
0,Alice,28,2023-07-21,3,Alice Engel
1,Sophia,24,2022-09-24,4,Sophia Engel
2,Júlia,20,2021-11-28,4,Júlia Engel


In [10]:
#Map: Only apply on series
df['AgeX2'] = df['Age'].map(lambda ag: ag * 2)
df.head()

,Name,Age,Date,Qtr,Fullname,AgeX2
0,Alice,28,2023-07-21,3,Alice Engel,56
1,Sophia,24,2022-09-24,4,Sophia Engel,48
2,Júlia,20,2021-11-28,4,Júlia Engel,40


In [11]:
#Applymap: Only apply on DataFrame
df['Year'] = df[['Date']].applymap(lambda dt: dt.year)
df.head()

/tmp/ipykernel_32140/2877058053.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df['Year'] = df[['Date']].applymap(lambda dt: dt.year)


,Name,Age,Date,Qtr,Fullname,AgeX2,Year
0,Alice,28,2023-07-21,3,Alice Engel,56,2023
1,Sophia,24,2022-09-24,4,Sophia Engel,48,2022
2,Júlia,20,2021-11-28,4,Júlia Engel,40,2021


In [12]:
#How to query pd.DataFrame using condition in df[df['col'] == val] or query('col == val') method:
#Ref: https://note.nkmk.me/en/python-pandas-query
#Case-01:
df[df['Qtr'] >= 4].head()

,Name,Age,Date,Qtr,Fullname,AgeX2,Year
1,Sophia,24,2022-09-24,4,Sophia Engel,48,2022
2,Júlia,20,2021-11-28,4,Júlia Engel,40,2021


In [13]:
#Case-02: using query('col == val') method
df.query('Qtr == 3').head()

,Name,Age,Date,Qtr,Fullname,AgeX2,Year
0,Alice,28,2023-07-21,3,Alice Engel,56,2023


In [14]:
#Case-02.1: to use variable in query string use @ in query-string
val = 3
df.query('Qtr <= @val').head()

,Name,Age,Date,Qtr,Fullname,AgeX2,Year
0,Alice,28,2023-07-21,3,Alice Engel,56,2023


In [15]:
#Case-02.2: or using f-strings as query
val = 3
df.query(f'Qtr <= {val}').head()

,Name,Age,Date,Qtr,Fullname,AgeX2,Year
0,Alice,28,2023-07-21,3,Alice Engel,56,2023


In [60]:
#Case-03: value isna() or notna() check in query string
df.query('Name.isna()').shape

(0, 7)

In [61]:
df.query('Name.notna()').shape

(3, 7)